<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#compare performance of different selection methods, crossovers, and mutations

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [1]:
import sys
sys.path.append('..')

In [ ]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import combinations
from typing import Callable
from library.algorithms.genetic_algorithms.algorithm import get_best_ind

In [3]:
# Import the relationship matrix as a pandas dataframe for visualization

relationship_matrix_df = pd.read_csv('data/seating_data.csv')
relationship_matrix_df.drop(columns='idx', inplace=True)
relationship_matrix_df

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


In [4]:
# Import the relationship matrix as a numpy array

relationship_matrix = np.loadtxt('data/seating_data.csv', delimiter=',', skiprows=1)
relationship_matrix = relationship_matrix[ : , 1:]
relationship_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]])

---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [5]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                #while any(idx in table for table in tables):
                    #idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    def fitness(self, verbose=False):
        total_happiness = 0
        i = 1
        for table in self.repr:
            # Sum relationship scores for all unique guest pairs at the table
            table_happiness = sum(self.relationship_matrix[a, b] for a, b in combinations(table, 2))
            total_happiness += table_happiness
            if verbose:
                print(f"Table {i} Happiness = {table_happiness}")
                i += 1
        return total_happiness

In [6]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness(verbose=True))

Random solution: [[27, 17, 21, 35, 48, 57, 26, 7], [58, 38, 59, 47, 34, 42, 15, 45], [0, 11, 18, 14, 4, 32, 43, 29], [12, 1, 24, 44, 6, 51, 36, 46], [41, 30, 9, 19, 16, 2, 25, 60], [62, 28, 37, 53, 10, 31, 55, 63], [54, 40, 50, 52, 8, 22, 33, 49], [20, 3, 39, 23, 5, 56, 13, 61]]
Table 1 Happiness = 3500.0
Table 2 Happiness = 0.0
Table 3 Happiness = 1700.0
Table 4 Happiness = 900.0
Table 5 Happiness = 1500.0
Table 6 Happiness = 1100.0
Table 7 Happiness = 700.0
Table 8 Happiness = -200.0
Total Fitness: 9200.0


---

### <span style="color:#96d9f0;">3. Selection Algorithms</span>

#### 3.1. Fitness Proportionate Selection

In a typical **fitness proportionate selection** (also known as **roulette wheel selection**), individuals are selected based on their fitness values, with higher fitness values having a greater chance of being chosen. In our optimization problem, however, the fitness values can range from **negative** (indicating conflicts) to **positive** (indicating good relationships between guests).

To adapt the selection process to handle **negative fitness values**, we introduce the following modifications:

1. **Handling Negative Fitness**:
   - Negative fitness values (e.g., -1000) represent undesirable relationships between guests and are valid in our case.
   - However, to ensure that these negative values don't completely block a solution from being selected, we **normalize** the fitness values by adding an **offset** to make all fitness values non-negative.
   - The offset is the **absolute value of the minimum fitness** if it's negative, ensuring that all fitness values become positive or zero without altering the relative fitness ranking of solutions.

2. **Fitness Normalization**:
   - For each individual, we add the **offset** (if necessary) to their fitness value to ensure it's non-negative.
   - This ensures that individuals with negative fitness still have a chance to be selected but are less likely to be chosen compared to those with higher (positive) fitness.

3. **Roulette Wheel Selection**:
   - After normalizing the fitness values, the selection process proceeds as usual, with higher fitness solutions having a higher probability of being selected.
   - By adding the offset, we make sure that **negative fitness values** don't result in zero or near-zero chances of selection, allowing them to contribute to the evolutionary process.

In [7]:
def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    if maximization:
        fitness_values = []
        min_fitness = min(ind.fitness() for ind in population)  # Find the minimum fitness value
        
        # Add a constant offset to ensure no negative fitness values
        offset = abs(min_fitness) if min_fitness < 0 else 0
        
        # Add the offset to each fitness value
        for ind in population:
            fitness_value = ind.fitness() + offset
            fitness_values.append(fitness_value)
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    
    # For each individual, check if random number is inside the individual's "box"
    box_boundary = 0
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)

In [8]:
population = [WSSolution() for _ in range(10)]

fitness_proportionate_selection_result = fitness_proportionate_selection(population, maximization=True)
print('Fitness Proportionate selection result:', fitness_proportionate_selection_result)

Fitness Proportionate selection result: [[3, 63, 4, 57, 2, 31, 35, 21], [40, 33, 18, 11, 46, 25, 41, 20], [51, 56, 1, 49, 24, 10, 22, 9], [59, 7, 12, 53, 61, 6, 17, 28], [0, 50, 34, 26, 44, 27, 37, 48], [14, 58, 19, 29, 8, 52, 32, 60], [23, 30, 45, 39, 42, 47, 15, 54], [16, 13, 55, 38, 62, 36, 5, 43]]


#### 3.2. Ranking Selection

With Ranking Selection, instead of selecting based on raw fitness values, individuals are ranked by their fitness, and higher-ranked individuals (better seating arrangements) have a higher chance of being selected. This method ensures that negative fitness values aren´t automatically excluded, as selection is based on relative rank rather than absolute fitness. It maintains diversity by giving lower-ranked individuals a chance to contribute, preventing premature convergence.

In [9]:
def ranking_selection(population, maximization=False):
    n_parents = 1  # Always select one parent per call in the GA

    # Sort population based on fitness
    sorted_pop = sorted(
        population,
        key=lambda sol: sol.fitness(),
        reverse=maximization  # reverse=True if maximizing
    )

    # Assign linear ranks
    n = len(sorted_pop)
    ranks = np.arange(1, n + 1)
    total_rank = np.sum(ranks)
    probabilities = ranks / total_rank

    # If maximizing, best is first → reverse probabilities so high rank = high prob
    if maximization:
        probabilities = probabilities[::-1]

    selected = np.random.choice(sorted_pop, size=n_parents, p=probabilities, replace=True)
    return selected[0]  # Return single selected parent


In [10]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")


Fitness of individuals in population:
Individual 0: Fitness = 13800.00
Individual 1: Fitness = 11700.00
Individual 2: Fitness = 18200.00
Individual 3: Fitness = 9300.00
Individual 4: Fitness = 10500.00
Individual 5: Fitness = 10100.00
Individual 6: Fitness = 13800.00
Individual 7: Fitness = 12200.00
Individual 8: Fitness = 12100.00
Individual 9: Fitness = 15000.00

Ranking selection result:
Selected 1: [[44, 52, 29, 15, 55, 41, 60, 37], [36, 46, 43, 39, 63, 8, 4, 1], [22, 54, 19, 18, 48, 17, 62, 42], [57, 5, 35, 21, 20, 38, 49, 12], [47, 27, 2, 32, 53, 59, 31, 10], [34, 24, 58, 28, 33, 56, 25, 50], [13, 30, 0, 14, 11, 61, 40, 23], [3, 9, 26, 6, 51, 45, 16, 7]], Fitness = 11700.00
Selected 2: [[23, 2, 63, 25, 4, 24, 15, 54], [56, 18, 29, 43, 17, 6, 16, 27], [33, 36, 31, 7, 10, 60, 42, 20], [41, 46, 21, 12, 35, 32, 45, 14], [11, 26, 44, 39, 9, 1, 62, 51], [5, 52, 28, 47, 49, 50, 48, 58], [61, 37, 59, 3, 40, 22, 30, 19], [8, 55, 0, 53, 34, 57, 13, 38]], Fitness = 12100.00


In this case, we can see that the same individual (2nd best in rank) was selected twice. This is okay and makes that it coud happen especially with the high rank and small population.

In [11]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")

Fitness of individuals in population:
Individual 0: Fitness = 15800.00
Individual 1: Fitness = 11300.00
Individual 2: Fitness = 12100.00
Individual 3: Fitness = 17400.00
Individual 4: Fitness = 15400.00
Individual 5: Fitness = 9000.00
Individual 6: Fitness = 12900.00
Individual 7: Fitness = 10700.00
Individual 8: Fitness = 5700.00
Individual 9: Fitness = 13200.00

Ranking selection result:
Selected 1: [[2, 30, 6, 49, 7, 62, 60, 19], [47, 40, 52, 16, 22, 56, 1, 10], [54, 13, 21, 27, 31, 61, 11, 48], [23, 14, 28, 45, 32, 24, 37, 59], [36, 44, 42, 43, 18, 33, 50, 55], [39, 53, 12, 3, 63, 57, 26, 35], [8, 20, 9, 51, 15, 25, 38, 29], [4, 0, 34, 41, 17, 58, 46, 5]], Fitness = 10700.00
Selected 2: [[41, 53, 10, 9, 49, 63, 17, 32], [13, 24, 37, 26, 16, 52, 62, 12], [4, 42, 11, 27, 3, 48, 20, 18], [58, 59, 46, 44, 25, 51, 23, 40], [19, 54, 28, 2, 8, 1, 7, 30], [29, 39, 14, 47, 60, 33, 56, 22], [55, 50, 38, 0, 61, 6, 43, 5], [36, 15, 21, 45, 35, 34, 57, 31]], Fitness = 11300.00


#### 3.3. Tournament Selection

The tournament selection randomly chooses 𝑘 individuals at random from the population, compares their fitnesses, and returns the best individual out of those to be a parent. 

In [12]:
def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

In [13]:
population = [WSSolution() for _ in range(10)]

tournament_selection_result = tournament_selection(population)
print('Tournament selection result:', tournament_selection_result)

Tournament selection result: [[28, 3, 53, 2, 31, 1, 13, 54], [60, 6, 44, 34, 11, 55, 9, 61], [22, 4, 41, 43, 33, 0, 40, 8], [38, 16, 62, 46, 21, 23, 24, 30], [42, 29, 59, 25, 39, 32, 27, 58], [35, 17, 20, 52, 37, 47, 18, 63], [36, 57, 56, 12, 49, 26, 14, 48], [10, 5, 50, 19, 7, 51, 45, 15]]


To make sure that our tournament_selection implementation truly picks the fittest individual, we wrapped it in a helper function, tournament_verification, which prints out every step of the process. This function is needed because the core algorithm samples competitors internally, not allowing us to check the candidates considered or their fitness except for the winner that is returned. In tournament_verification, we set 
k=10 and explicitly list all ten randomly chosen competitors along with their fitness values, then identify and print the one selected as the fittest by tournament_selection, allowing us to directly compare that result against the ten and confirm it truly has the highest fitness. This gives us clear, reproducible evidence that the tournament selection returns the strongest contender.

In [14]:
def tournament_verification(population, k=10):
    competitors = random.sample(population, k)
    
    print("Fitness of competitors:")
    for i in range(k):
        print(f"Competitor {i+1}: Fitness = {competitors[i].fitness()}")
    
    best_idx, best_competitor = max(enumerate(competitors),key=lambda pair: pair[1].fitness())
    
    print("\nTournament selection result:")
    print(f"Competitor {best_idx+1}: Fitness = {best_competitor.fitness():.2f}")

In [15]:
population = [WSSolution() for _ in range(50)]

tournament_verification(population)

Fitness of competitors:
Competitor 1: Fitness = 14100.0
Competitor 2: Fitness = 19500.0
Competitor 3: Fitness = 13300.0
Competitor 4: Fitness = 8900.0
Competitor 5: Fitness = 12100.0
Competitor 6: Fitness = 15200.0
Competitor 7: Fitness = 11000.0
Competitor 8: Fitness = 10800.0
Competitor 9: Fitness = 18200.0
Competitor 10: Fitness = 15200.0

Tournament selection result:
Competitor 2: Fitness = 19500.00


---

### <span style="color:#96d9f0;">4. Crossover Operators</span>

#### 4.1. Custom Crossover Operator (`table_level_crossover`)

In this section, we implement a custom crossover operator tailored to the constraints of our optimization problem. Unlike standard one-point crossover, which is suitable for flat representations, our solution requires a structured approach:

- Each solution is a nested list representing 8 tables with 8 unique guests each.
- Every guest (from 0 to 63) must appear exactly once across all tables.
- A direct crossover risks creating invalid offspring (with duplicated or missing guests).

To address this, our operator performs table-level crossover:

1. A one-point crossover is applied between the two parent solutions at the table level. This can result in invalid intermediate offspring that contain duplicate guests and omit others, violating the uniqueness constraint.

2. The resulting child is then repaired by:

    - Detecting duplicate and missing guests
    - Replacing duplicates with missing ones to restore a valid configuration
    - This ensures all offspring are valid solutions and suitable for use in the genetic algorithm.

In [16]:
def table_level_crossover(parent1, parent2):
    # Deep copies of the parent representations
    p1_repr = deepcopy(parent1.repr)
    p2_repr = deepcopy(parent2.repr)

    # Flatten guest lists
    total_guests = len(parent1.relationship_matrix)

    # One-point crossover at table level
    xo_point = random.randint(1, len(p1_repr) - 1)
    child1_repr = p1_repr[:xo_point] + p2_repr[xo_point:]
    child2_repr = p2_repr[:xo_point] + p1_repr[xo_point:]

    def repair(child_repr):
        # Count how many times each guest appears
        guest_counts = {}
        for table in child_repr:
            for guest in table:
                guest_counts[guest] = guest_counts.get(guest, 0) + 1

        # Identify missing and extra guests
        missing = [g for g in range(total_guests) if guest_counts.get(g, 0) == 0]
        extras = [g for g, count in guest_counts.items() if count > 1 for _ in range(count - 1)]

        # Replace extra guests with missing ones
        for table in child_repr:
            for i in range(len(table)):
                guest = table[i]
                if guest_counts[guest] > 1:
                    new_guest = missing.pop()
                    guest_counts[guest] -= 1
                    table[i] = new_guest
                    guest_counts[new_guest] = 1  # added to the table

        return child_repr

    # Repair both children
    repaired1 = repair(child1_repr)
    repaired2 = repair(child2_repr)

    return WSSolution(repr=repaired1), WSSolution(repr=repaired2)

In [17]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = table_level_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[33, 22, 53, 29, 30, 27, 20, 37], [39, 18, 31, 17, 35, 8, 0, 56], [44, 34, 63, 54, 51, 46, 21, 1], [5, 6, 47, 11, 3, 38, 45, 49], [2, 40, 41, 60, 58, 10, 13, 4], [15, 48, 61, 7, 24, 57, 52, 43], [62, 16, 26, 28, 32, 25, 42, 50], [36, 19, 12, 14, 9, 23, 55, 59]]
Parent 2 Seating Arrangement:
[[27, 28, 24, 57, 31, 48, 3, 15], [17, 14, 37, 36, 61, 20, 46, 49], [23, 11, 2, 43, 55, 51, 25, 1], [44, 33, 5, 62, 22, 6, 9, 63], [38, 50, 41, 19, 16, 7, 4, 30], [40, 56, 45, 39, 54, 0, 60, 18], [21, 32, 13, 34, 42, 8, 52, 59], [10, 12, 35, 29, 58, 26, 47, 53]]

Offspring 1 Seating Arrangement:
[[61, 57, 49, 48, 46, 27, 20, 37], [36, 28, 31, 17, 24, 15, 14, 3], [23, 11, 2, 43, 55, 51, 25, 1], [44, 33, 5, 62, 22, 6, 9, 63], [38, 50, 41, 19, 16, 7, 4, 30], [40, 56, 45, 39, 54, 0, 60, 18], [21, 32, 13, 34, 42, 8, 52, 59], [10, 12, 35, 29, 58, 26, 47, 53]]
Offspring 2 Seating Arrangement:
[[27, 56, 53, 39, 31, 35, 33, 30], [17, 29, 37, 22, 18, 20, 8, 0], [44, 34, 63, 54, 

In [18]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.2. Partially Matched Crossover

The Partially Matched Crossover (pmc) is a genetic algorithm operator designed for permutations, like our guest seating arrangement. It swaps a segment between two parent solutions and builds offspring by preserving the order and position of elements while avoiding duplicates. In our implementation, we adapted PMX to work with the wedding seating format by flattening table arrangements into 1D lists, applying PMX, and then converting them back. We carefully resolved conflicts during crossover by mapping duplicate entries to valid unused guests, ensuring that each guest appears exactly once in the final seating—maintaining valid, non-repetitive arrangements.

In [19]:
def partially_matched_crossover(parent1, parent2):
    """
    PMC crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    import random

    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def pmc(p1, p2):
        child = [None] * size
        # Copy the crossover segment from p1
        child[point1:point2] = p1[point1:point2]

        for i in range(point1, point2):
            if p2[i] not in child:
                val = p2[i]
                idx = i
                while True:
                    mapped = p1[idx]
                    idx = p2.index(mapped)
                    if child[idx] is None:
                        child[idx] = val
                        break

        # Fill remaining positions from p2
        for i in range(size):
            if child[i] is None:
                child[i] = p2[i]

        return child

    child1_flat = pmc(parent1_flat, parent2_flat)
    child2_flat = pmc(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2


In [20]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = partially_matched_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[34, 18, 44, 47, 15, 63, 28, 50], [29, 58, 26, 9, 13, 37, 1, 52], [23, 36, 30, 35, 8, 2, 45, 33], [7, 14, 11, 49, 62, 4, 38, 27], [0, 17, 24, 19, 32, 12, 39, 53], [42, 57, 3, 16, 6, 20, 40, 21], [43, 56, 61, 48, 46, 5, 25, 51], [22, 31, 55, 60, 54, 41, 10, 59]]
Parent 2 Seating Arrangement:
[[30, 7, 5, 57, 10, 56, 13, 43], [47, 54, 52, 1, 14, 20, 34, 24], [17, 33, 48, 46, 21, 50, 31, 29], [37, 16, 19, 28, 45, 36, 41, 4], [6, 55, 8, 15, 62, 9, 60, 53], [38, 51, 63, 61, 39, 44, 58, 12], [42, 0, 23, 40, 3, 26, 49, 27], [22, 32, 2, 11, 25, 35, 18, 59]]

Offspring 1 Seating Arrangement:
[[54, 44, 34, 47, 10, 60, 28, 50], [29, 58, 26, 9, 13, 37, 1, 52], [23, 36, 30, 35, 8, 2, 45, 33], [7, 14, 11, 49, 62, 4, 38, 27], [0, 17, 24, 19, 32, 12, 39, 53], [42, 57, 3, 16, 6, 20, 40, 21], [43, 56, 61, 48, 46, 5, 25, 51], [22, 31, 41, 15, 55, 63, 18, 59]]
Offspring 2 Seating Arrangement:
[[5, 18, 7, 57, 11, 35, 13, 43], [47, 54, 52, 1, 14, 20, 34, 24], [17, 33, 48, 46, 2

In [21]:
# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.3. Order Crossover

The Order Crossover (OX) is a genetic algorithm operator also designed for permutation problems, such as our guest seating arrangements. It selects a contiguous block of seats from one parent and preserves both the relative order and positional integrity of the remaining guests by filling in, in sequence, from the other parent’s list and skipping any guest already included. In our implementation, we adapt OX to the wedding format by flattening each table layout into a single list, choosing two crossover points, copying the segment from Parent 1 into Offspring 1 (and vice versa for Offspring 2), then rotating through the other parent’s flattened list to fill the empty slots while avoiding duplicates. Finally, we reshape the 1D offspring lists back into the original table structure, guaranteeing a complete, non-repetitive seating plan.

In [ ]:
def order_crossover(parent1, parent2):
    """
    Order crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def order(p1, p2):
        child = [None] * size
        child[point1:point2] = p1[point1:point2]
        
        for i in range(point2-size, point1):
            val = p2[i]
            idx = i
            while val in child:
                idx+=1
                val = p2[idx]
            child[i] = val

        return child

    child1_flat = order(parent1_flat, parent2_flat)
    child2_flat = order(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2

In [23]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply Order crossover
offspring1, offspring2 = order_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())

Parent 1 Seating Arrangement:
[[44, 41, 61, 35, 18, 7, 1, 9], [45, 26, 15, 17, 30, 52, 34, 42], [36, 28, 8, 53, 13, 14, 12, 48], [10, 47, 21, 46, 43, 32, 2, 27], [6, 38, 57, 37, 39, 11, 50, 49], [25, 19, 59, 54, 3, 20, 56, 62], [58, 60, 4, 0, 33, 51, 31, 5], [23, 22, 16, 29, 63, 24, 40, 55]]
Parent 2 Seating Arrangement:
[[38, 31, 60, 9, 30, 45, 47, 22], [53, 17, 18, 4, 2, 41, 36, 28], [49, 63, 57, 20, 25, 33, 44, 51], [1, 34, 40, 29, 58, 54, 5, 6], [50, 59, 27, 3, 12, 37, 43, 32], [26, 14, 48, 11, 55, 8, 15, 52], [23, 35, 56, 61, 0, 42, 19, 62], [39, 21, 7, 10, 24, 46, 13, 16]]

Offspring 1 Seating Arrangement:
[[63, 57, 20, 25, 33, 44, 51, 1], [40, 26, 15, 17, 30, 52, 34, 42], [36, 28, 8, 53, 13, 14, 12, 48], [10, 47, 21, 29, 58, 54, 5, 6], [50, 59, 27, 3, 37, 43, 32, 11], [55, 23, 35, 56, 61, 0, 19, 62], [39, 7, 24, 46, 16, 38, 31, 60], [9, 45, 22, 18, 4, 2, 41, 49]]
Offspring 2 Seating Arrangement:
[[8, 53, 13, 14, 12, 48, 10, 47], [21, 17, 18, 4, 2, 41, 36, 28], [49, 63, 57, 20, 2

In [24]:
# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


---

### <span style="color:#96d9f0;">5. Mutation Operators</span>

#### 5.1. Swap Mutation

The swap mutation operator introduces random changes to the seating arrangement by swapping two guests between different tables.

It's important to note that swapping two guests within the same table would not create any new configuration - it would simply result in the same solution as before. Since we want to introduce meaningful changes, the swap mutation ensures that the two guests are always swapped between different tables. This prevents the mutation from generating an invalid or identical solution.

This mutation helps the genetic algorithm explore new seating arrangements by shuffling guests across tables while respecting the overall constraints of the problem.

In [25]:
def swap_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    if len(tables) < 2:
        return new_solution

    # Pick two random tables
    t1, t2 = random.sample(range(len(tables)), 2)
    if not tables[t1] or not tables[t2]:
        return new_solution

    # Swap one person from each
    i1 = random.randint(0, len(tables[t1]) - 1)
    i2 = random.randint(0, len(tables[t2]) - 1)
    tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

    return new_solution

In [26]:
solution = WSSolution()

mutated_solution = swap_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[8, 34, 40, 33, 13, 9, 62, 55], [53, 48, 5, 59, 49, 29, 43, 16], [21, 54, 24, 18, 19, 0, 51, 41], [31, 58, 56, 63, 35, 3, 14, 52], [28, 44, 25, 45, 10, 47, 1, 11], [60, 61, 20, 12, 17, 4, 22, 2], [57, 38, 42, 15, 30, 32, 37, 50], [6, 7, 27, 26, 46, 39, 23, 36]]
Mutated solution: [[8, 34, 40, 33, 13, 9, 62, 55], [53, 48, 5, 59, 49, 29, 14, 16], [21, 54, 24, 18, 19, 0, 51, 41], [31, 58, 56, 63, 35, 3, 43, 52], [28, 44, 25, 45, 10, 47, 1, 11], [60, 61, 20, 12, 17, 4, 22, 2], [57, 38, 42, 15, 30, 32, 37, 50], [6, 7, 27, 26, 46, 39, 23, 36]]


#### 5.2. Inversion Mutation

The inversion mutation operator introduces variation by selecting a random segment of the seating plan and reversing the order of guests within that segment.

Since the seating arrangement is represented as a list of tables, the entire structure is first flattened into a single list containing all guests. The inversion is then performed on a random sublist, effectively inverting a continuous sequence of guests. After the inversion, the list is reshaped back into the original table format.

This mutation allows the algorithm to explore new configurations by reordering guests while preserving the structure of the solution. To ensure validity, a repair function is applied after mutation to guarantee that all guests remain uniquely assigned with no duplicates or omissions.

In [ ]:
def inversion_mutation_seating(individual: WSSolution, mut_prob):
    # Access the seating arrangement (8 tables with guests)
    tables = individual.repr

    rows = len(tables)
    cols = len(tables[0])
    
    # Flatten the seating arrangement into a single list of guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation with the given probability
    if random.random() < mut_prob:
        # Select two random indices for the inversion
        start_row, end_row = sorted(random.sample(range(rows), 2))
        start_col, end_col = random.sample(range(cols), 2)
        start = start_row * cols + start_col
        end = end_row * cols + end_col
        # Reverse the segment between the two indices
        flat[start:end + 1] = reversed(flat[start:end + 1])

    # Reshape the flat list back into 8 tables of 8 guests each
    new_tables = [flat[i:i + individual.table_capacity] for i in range(0, len(flat), individual.table_capacity)]

    # Return a new WSSolution with the mutated seating arrangement
    return WSSolution(repr=new_tables, relationship_matrix=individual.relationship_matrix, nr_of_tables=individual.nr_of_tables)

In [ ]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[61, 50, 52, 11, 15, 39, 7, 21], [19, 58, 62, 29, 40, 10, 38, 31], [32, 18, 53, 36, 35, 56, 25, 55], [26, 17, 4, 34, 49, 33, 37, 57], [0, 23, 63, 5, 48, 22, 54, 30], [46, 45, 51, 14, 60, 8, 3, 20], [27, 44, 13, 2, 28, 16, 12, 1], [43, 9, 6, 41, 47, 24, 59, 42]]
Mutated solution: [[61, 50, 52, 11, 15, 39, 7, 21], [19, 58, 62, 29, 40, 10, 38, 31], [5, 63, 23, 0, 57, 37, 33, 49], [34, 4, 17, 26, 55, 25, 56, 35], [36, 53, 18, 32, 48, 22, 54, 30], [46, 45, 51, 14, 60, 8, 3, 20], [27, 44, 13, 2, 28, 16, 12, 1], [43, 9, 6, 41, 47, 24, 59, 42]]


#### 5.3. Scramble Mutation

The scramble mutation operator introduces variation by selecting a random subset of guests in the seating plan and scrambling their order within the selected segment. Unlike the inversion mutation, where the order is reversed, the scramble mutation randomizes the order of guests in a selected sublist.

The process begins by flattening the seating arrangement, which is represented as a list of tables, into a single list of guests. Then, a random segment of guests is selected, and the order of the guests within that segment is scrambled. After the scrambling is performed, the list is reshaped back into the original table structure.

This mutation operator allows the algorithm to explore different configurations by rearranging guests within a specified segment of the seating arrangement, offering a broader search of potential solutions. It maintains the structure of the solution by preserving the overall number of guests per table while allowing the order of guests to vary.

To ensure validity and correctness of the solution, a repair function is applied after the mutation to guarantee that all guests remain uniquely assigned, with no duplicates or omissions.

In [29]:
def scramble_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    rows = len(tables)
    cols = len(tables[0])

    if rows < 2:
        return new_solution

    selected_cols = [random.randint(0, cols-1) for _ in range(rows)]
    selected_positions = [(row, selected_cols[row]) for row in range(rows)]

    selected_values = []
    for row, col in selected_positions: 
        if not tables[row][col]:
            return new_solution
        else: 
            selected_values.append(tables[row][col])
    
    def deranged_shuffle(lst):
        while True:
            shuffled = lst[:] 
            random.shuffle(shuffled)
            if all(a != b for a, b in zip(lst, shuffled)):
                return shuffled
    
    selected_values = deranged_shuffle(selected_values)

    for (row, col), value in zip(selected_positions, selected_values):
        tables[row][col] = value

    return new_solution

In [30]:
solution = WSSolution()

mutated_solution = scramble_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[28, 10, 7, 62, 43, 30, 41, 23], [37, 32, 3, 56, 52, 24, 58, 15], [1, 22, 46, 25, 57, 50, 11, 36], [55, 63, 44, 2, 49, 47, 20, 60], [9, 39, 18, 8, 34, 12, 51, 38], [19, 0, 48, 61, 14, 17, 27, 33], [53, 21, 59, 45, 5, 35, 31, 26], [40, 4, 13, 29, 54, 6, 42, 16]]
Mutated solution: [[28, 45, 7, 62, 43, 30, 41, 23], [37, 32, 61, 56, 52, 24, 58, 15], [18, 22, 46, 25, 57, 50, 11, 36], [55, 63, 44, 2, 49, 47, 16, 60], [9, 39, 3, 8, 34, 12, 51, 38], [19, 0, 48, 20, 14, 17, 27, 33], [53, 21, 59, 10, 5, 35, 31, 26], [40, 4, 13, 29, 54, 6, 42, 1]]


### <span style="color:#96d9f0;">6. Random Trials</span>

In [38]:
import time

start = time.time()
for i in range(10):
    res = 0
    scores = []
    for j in range(100000):
        solution = WSSolution()
        res=solution.fitness()
        scores.append(res)
    avg_score = sum(scores)/100000
    best_score = max(scores)
    print(f"Result {i+1}: Average Fitness: {avg_score} and Best Fitness Found {best_score}")

end = time.time()

print(f"Time elapsed: {int((end- start)//60)} minutes and {round((end-start)%60)} seconds")

Result 1: Average Fitness: 12196.453 and Best Fitness Found 30900.0
Result 2: Average Fitness: 12201.305 and Best Fitness Found 30600.0
Result 3: Average Fitness: 12210.007 and Best Fitness Found 30800.0
Result 4: Average Fitness: 12202.181 and Best Fitness Found 29900.0
Result 5: Average Fitness: 12200.442 and Best Fitness Found 32500.0
Result 6: Average Fitness: 12209.278 and Best Fitness Found 30900.0
Result 7: Average Fitness: 12208.056 and Best Fitness Found 34400.0
Result 8: Average Fitness: 12212.482 and Best Fitness Found 32700.0
Result 9: Average Fitness: 12206.165 and Best Fitness Found 33600.0
Result 10: Average Fitness: 12204.035 and Best Fitness Found 31500.0
Time elapsed: 2 minutes and 15 seconds


If 1000000 solutions took around 2 minutes, then the total seating arrangements (assuming a much simpler scenario where all random solutions displayed never repeat) would be:

In [4]:
import math

total_combinations = (math.comb(64,8)*math.comb(56,8)*math.comb(48, 8)*math.comb(40, 8)*math.comb(32, 8)*math.comb(24, 8)*math.comb(16,8)) / math.factorial(8)
minutes_taken = total_combinations * 2 / 1000000
years_taken = minutes_taken/(60*24*365) 


print(f" There are {total_combinations} different seating arrangements and it would take around {years_taken} years for a computer to compare all the solutions and find the best.")

 There are 4.505387879868752e+47 different seating arrangements and it would take around 1.7143789497217472e+36 years for a computer to compare all the solutions and find the best.


---

### <span style="color:#96d9f0;">7. Genetic Algorithm</span>

In [33]:
def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    crossover_operator: Callable,
    mutation_operator: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes a genetic algorithm to optimize a population of solutions.

    Args:
        initial_population (list[Solution]): The starting population of solutions.
        max_gen (int): The maximum number of generations to evolve.
        selection_algorithm (Callable): Function used for selecting individuals.
        crossover_operator (Callable): Function to perform crossover between two individuals.
        mutation_operator (Callable): Function to mutate an individual.
        maximization (bool, optional): If True, maximizes the fitness function; otherwise, minimizes. Defaults to False.
        xo_prob (float, optional): Probability of applying crossover. Defaults to 0.9.
        mut_prob (float, optional): Probability of applying mutation. Defaults to 0.2.
        elitism (bool, optional): If True, carries the best individual to the next generation. Defaults to True.
        verbose (bool, optional): If True, prints detailed logs for debugging. Defaults to False.

    Returns:
        Solution: The best solution found after evolving for max_gen generations.
    """
    population = initial_population

    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        new_population = []

        if elitism:
            new_population.append(deepcopy(get_best_ind(population, maximization)))

        while len(new_population) < len(population):
            parent1 = selection_algorithm(population, maximization)
            parent2 = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected parents:\n{parent1}\n{parent2}')

            if random.random() < xo_prob:
                offspring1, offspring2 = crossover_operator(parent1, parent2)
                if verbose:
                    print('Crossover applied.')
            else:
                offspring1, offspring2 = deepcopy(parent1), deepcopy(parent2)
                if verbose:
                    print('Replication applied.')

            mutated1 = mutation_operator(offspring1, mut_prob)
            new_population.append(mutated1)
            if verbose:
                print(f'Mutated offspring 1: {mutated1}')

            if len(new_population) < len(population):
                mutated2 = mutation_operator(offspring2, mut_prob)
                new_population.append(mutated2)
                if verbose:
                    print(f'Mutated offspring 2: {mutated2}')

        population = new_population

        if verbose:
            best = get_best_ind(population, maximization)
            print(f'Best individual fitness this generation: {best.fitness()}')

    return get_best_ind(population, maximization)

In [32]:
# Number of individuals in the initial population
population_size = 50

# Generate initial population of random WSSolutions
population = [WSSolution() for _ in range(population_size)]

In [34]:
best = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best)
print("Fitness:", best.fitness())

In [35]:
isa_try = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(isa_try)
print("Fitness:", isa_try.fitness())

In [36]:
#Number of individuals in the initial population
population_size2 = 3

# Generate initial population of random WSSolutions
population2 = [WSSolution() for _ in range(population_size2)]

best2 = genetic_algorithm(
    initial_population=population2,
    max_gen=5,
    selection_algorithm=tournament_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=inversion_mutation_seating,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best2)
print("Fitness:", best2.fitness())

In [37]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
pmc_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(pmc_test.repr)
print("Fitness:", pmc_test.fitness())

In [62]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(pmc_test.repr))

Offspring 2 has no repeated guests: True


In [38]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
ranking_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(ranking_test.repr)
print("Fitness:", ranking_test.fitness())

In [61]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(ranking_test.repr))

Offspring 2 has no repeated guests: True
